# Thinkful Data Science Course

#Unit 3: Answering Questions with Data

##Lesson 2: Cloudy with a Chance of Data - Analyzing Temperature Data

####CHALLENGE
Build the API call by combining the string elements in Python for your first city. You can use the datetime.datetime.now() function from the datetime package for the current datetime. You can use the datetime.timedelta() function to subtract or add time to a date. In this case, we'll be subtracting 30 days from the current date to get our start date and then iterating through until the present day. We do that like this start_date = datetime.datetime.now() - datetime.timedelta(days=30). This will subtract 30 days from the current day.

In [5]:
import requests
import sqlite3 as lite
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import collections

Test the call for your first city and make sure you have it formatted properly. You can start by just printing out the URL and pasting it into your browser before you use the requests package to do the call for you. This can help you troubleshoot any errors (though you can use the text and status_code attributes to also troubleshoot any errors)

In [5]:
api_key = "66a4ee1dd841976fcf36a94a3b3b930e"
url = 'https://api.forecast.io/forecast/' + api_key + "/"
cities = { "Minneapolis": '44.963324,-3.268320',
            "San_Francisco": '37.727239,-123.032229',
            "Sioux_Falls": '43.550116, -96.729280',
            "Tipton": '42.015974, -84.064363',
            "Pheonix": '33.572154,-112.090132'
            # "Abu Dhabi": '24.483103, 54.353026',
            # "Fehmarn": '54.465705, 11.136710'
        }

Once you have the URL formatted properly, issue the request from your code and inspect the result. How many levels does the data have? Which field do we want to save to get the daily maximum temperature?

In [6]:
end_date = datetime.datetime.now()
query_date = end_date - datetime.timedelta(days=30) #the current value being processed

In [7]:
con = lite.connect('weather.db')
cur = con.cursor()


In [8]:
cities.keys()

dict_keys(['Tipton', 'Sioux_Falls', 'Pheonix', 'Minneapolis', 'San_Francisco'])

Based on the data sample, create the table in a SQLite database called "weather.db".

In [9]:
with con:
    cur.execute("DROP TABLE IF EXISTS daily_temp")
    cur.execute('CREATE TABLE daily_temp ( day_of_reading INT, Minneapolis REAL, San_Francisco REAL, Sioux_Falls REAL, Tipton REAL, Pheonix REAL);') #use your own city names instead of city1...

In SQL, a row has to be inserted before it can be updated. In order to keep the code clean, we're going to iterate through the values in the range and insert them into the database without any other values

In [10]:
with con:
    while query_date < end_date:
        cur.execute("INSERT INTO daily_temp(day_of_reading) VALUES (?)", (int(query_date.strftime('%s')),))
        query_date += datetime.timedelta(days=1)

Now we can loop through our cities and query the API

Write a script that takes each city and queries every day for the past 30 days (Hint: You can use the datetime.timedelta(days=1) to increment the value by day)

Save the max temperature values to the table, keyed on the date. You can query for the value

In [11]:
for k,v in cities.items():
    query_date = end_date - datetime.timedelta(days=30) #set value each time through the loop of cities
    while query_date < end_date:
        #query for the value
        r = requests.get(url + v + ',' +  query_date.strftime("%Y-%m-%dT%H:%M:%S"))

        with con: #inrsert the temperature max to the database
            cur.execute('UPDATE daily_temp SET ' + k + ' = ' + str(r.json()['daily']['data'][0]['temperatureMax']) + ' WHERE day_of_reading = ' + query_date.strftime('%s'))
        #increment query_date to the next day for next operation of loop
        query_date += datetime.timedelta(days=1) #increment query_date to the next day

con.close()

In [27]:
con = lite.connect('weather.db')
cur = con.cursor()

df = pd.read_sql_query("SELECT * FROM daily_temp ORDER BY day_of_reading", con, index_col='day_of_reading')

In [28]:
df

,Minneapolis,San_Francisco,Sioux_Falls,Tipton,Pheonix
day_of_reading,,,,,
1438707734,71.21,65.51,77.94,79.18,108.81
1438794134,77.46,62.70,79.00,77.89,109.36
1438880534,74.30,60.90,81.60,75.72,102.99
1438966934,71.47,63.51,87.86,77.20,103.48
1439053334,66.57,62.51,83.60,76.10,99.94
1439139734,70.62,61.70,83.46,79.28,104.18
1439226134,75.87,63.51,81.13,75.40,102.28
1439312534,82.26,62.21,85.19,79.23,95.84
1439398934,75.73,66.11,88.26,75.62,105.61


###Profiling Temperature Data

Follow the example in the previous lesson and in the lessons from the last unit and describe the data you have. What's the range of temperatures for each city? Whatis the mean temperature for each city? What's the variance? Are there any patterns in the data? Which cities had the largest temperature changes over the time period?

####Range

In [6]:
print('In Minneapolis, the temperature has ranged from ', min(df['Minneapolis']), 'to ', max(df['Minneapolis']))
print('In San Francisco, the temperature has ranged from ', min(df['San_Francisco']), 'to ', max(df['San_Francisco']))
print('In Sioux Falls, the temperature has ranged from ', min(df['Sioux_Falls']), 'to ', max(df['Sioux_Falls']))
print('In Tipton, the temperature has ranged from ', min(df['Tipton']), 'to ', max(df['Tipton']))
print('In Phoenix, the temperature has ranged from ', min(df['Pheonix']), 'to ', max(df['Pheonix']))
# print('In Abu Dhabi, the temperature has ranged from ', min(df['Abu_Dhabi']), 'to ', max(df['Abu_Dhabi']))
# print('In Fehmarn, the temperature has ranged from ', min(df['Fehmarn']), 'to ', max(df['Fehmarn']))

In Minneapolis, the temperature has ranged from  66.05 to  82.26
In San Francisco, the temperature has ranged from  60.11 to  66.11
In Sioux Falls, the temperature has ranged from  66.45 to  90.53
In Tipton, the temperature has ranged from  63.86 to  88.35
In Phoenix, the temperature has ranged from  95.84 to  113.15


In [60]:
print('In Minneapolis, the temperature ranged over ', max(df['Minneapolis']) - min(df['Minneapolis']), 'degrees')
print('In San Francisco, the temperature ranged over ', max(df['San_Francisco']) - min(df['San_Francisco']), 'degrees')
print('In Sioux Falls, the temperature ranged over ', max(df['Sioux_Falls']) - min(df['Sioux_Falls']), 'degrees')
print('In Tipton, the temperature ranged over ', max(df['Tipton']) - min(df['Tipton']), 'degrees')
print('In Phoenix, the temperature ranged over ', max(df['Pheonix']) - min(df['Pheonix']), 'degrees')
# print('In Abu Dhabi, the temperature has ranged from ', min(df['Abu_Dhabi']), 'to ', max(df['Abu_Dhabi']))
# print('In Fehmarn, the temperature has ranged from ', min(df['Fehmarn']), 'to ', max(df['Fehmarn']))

In Minneapolis, the temperature ranged over  16.21 degrees
In San Francisco, the temperature ranged over  6.0 degrees
In Sioux Falls, the temperature ranged over  24.08 degrees
In Tipton, the temperature ranged over  24.49 degrees
In Phoenix, the temperature ranged over  17.31 degrees


####Mean

In [7]:
print('In Minneapolis, the mean temperature from the past thirty days is', (df['Minneapolis'].mean()))
print('In San Francisco, the mean temperature from the past thirty days is', (df['San_Francisco'].mean()))
print('In Sioux Fals, the mean temperature from the past thirty days is', (df['Sioux_Falls'].mean()))
print('In Tipton, the mean temperature from the past thirty days is', (df['Tipton'].mean()))
print('In Phoenix, the mean temperature from the past thirty days is', (df['Pheonix'].mean()))
# print('In Abu Dhabi, the mean temperature from the past thirty days is', (df['Abu_Dhabi'].mean()))
# print('In Fehmarn, the mean temperature from the past thirty days is', (df['Fehmarn'].mean()))

In Minneapolis, the mean temperature from the past thirty days is 71.57799999999997
In San Francisco, the mean temperature from the past thirty days is 62.895333333333355
In Sioux Fals, the mean temperature from the past thirty days is 79.15066666666667
In Tipton, the mean temperature from the past thirty days is 78.49466666666665
In Phoenix, the mean temperature from the past thirty days is 104.32133333333331


####Variance

In [13]:
print('In Minneapolis, the variance in temperature from the past thirty days is', (df['Minneapolis'].var()))
print('In San Francisco, the variance in temperature from the past thirty days is', (df['San_Francisco'].var()))
print('In Sioux Falls, the variance in temperature from the past thirty days is', (df['Sioux_Falls'].var()))
print('In Tipton, the mean variance in from the past thirty days is', (df['Tipton'].var()))
print('In Phoenix, the mean variance in from the past thirty days is', (df['Pheonix'].var()))
# print('In Abu Dhabi, the mean temperature from the past thirty days is', (df['Abu_Dhabi'].mean()))
# print('In Fehmarn, the mean temperature from the past thirty days is', (df['Fehmarn'].mean()))

In Minneapolis, the variance in temperature from the past thirty days is 12.854754482758628
In San Francisco, the variance in temperature from the past thirty days is 1.8397636781609201
In Sioux Falls, the variance in temperature from the past thirty days is 44.90908919540232
In Tipton, the mean variance in from the past thirty days is 33.580474022988504
In Phoenix, the mean variance in from the past thirty days is 18.76946712643679


In [ ]:
plt.figure()
(df['Minneapolis']).hist()
plt.title('Temperatures in Minneapolis over last 30 days')
plt.xlabel('Temp (F)')
plt.ylabel('Count')
plt.show()

plt.close()
plt.figure()
(df['San_Francisco']).hist()
plt.title('Temperatures in San Francisco over last 30 days')
plt.xlabel('Temp (F)')
plt.ylabel('Count')
plt.show()

###Analyzing Temperature Data

Just as you did with the Citi Bike data, go through the the sequence of temperature readings and calculate the change between days and find the greatest range in high temperatures in the month you measured. Which city had the greatest variation? What is the distribution of the difference? Does the result surprise you? Why or why not?

Using this approach, we don't have to limit ourselves to 5 cities or analyzing only high temperatures. We could record all the weather variables and compare the changes over time for 100 cities in whatever timeframe we wanted (so long as we don't mind paying for the API calls). For a project that size, it might make sense to graduate from SQLite to a more fully featured database that could handle the extra load, but the principles stay the same.

In [6]:
day_change = collections.defaultdict(int)

In [7]:
day_change

defaultdict(<class 'int'>, {})

In [29]:
for col in df.columns:
    print(df[col].tolist())

[71.209999999999994, 77.459999999999994, 74.299999999999997, 71.469999999999999, 66.569999999999993, 70.620000000000005, 75.870000000000005, 82.260000000000005, 75.730000000000004, 71.079999999999998, 67.489999999999995, 68.609999999999999, 70.370000000000005, 71.810000000000002, 71.379999999999995, 72.049999999999997, 75.519999999999996, 74.560000000000002, 71.359999999999999, 69.620000000000005, 68.590000000000003, 70.879999999999995, 71.540000000000006, 74.349999999999994, 70.239999999999995, 73.069999999999993, 68.819999999999993, 68.209999999999994, 66.049999999999997, 66.25]
[65.510000000000005, 62.700000000000003, 60.899999999999999, 63.509999999999998, 62.509999999999998, 61.700000000000003, 63.509999999999998, 62.210000000000001, 66.109999999999999, 63.909999999999997, 64.0, 63.93, 64.120000000000005, 60.109999999999999, 62.210000000000001, 63.409999999999997, 62.409999999999997, 61.07, 61.200000000000003, 62.689999999999998, 63.25, 60.729999999999997, 61.68, 63.68999999999999

The following code will find the day-to-day changes in temperature

In [58]:

for col in df.columns:
    temp_id = col[:]
    print('The Temp ids are:', temp_id)
    temp_vals = df[col].tolist()
    print('The temp vals are: ',temp_vals)
    temp_change = 0

    for k,v in enumerate(temp_vals):
        if k < len(temp_vals) - 1:
            temp_change = abs(temp_vals[k] - temp_vals[k+1])
            print('The temp_change is: ',temp_change)
        
    day_change[(temp_id)] = temp_change 
    print('The day_change is:',day_change)

The Temp ids are: Minneapolis
The temp vals are:  [71.209999999999994, 77.459999999999994, 74.299999999999997, 71.469999999999999, 66.569999999999993, 70.620000000000005, 75.870000000000005, 82.260000000000005, 75.730000000000004, 71.079999999999998, 67.489999999999995, 68.609999999999999, 70.370000000000005, 71.810000000000002, 71.379999999999995, 72.049999999999997, 75.519999999999996, 74.560000000000002, 71.359999999999999, 69.620000000000005, 68.590000000000003, 70.879999999999995, 71.540000000000006, 74.349999999999994, 70.239999999999995, 73.069999999999993, 68.819999999999993, 68.209999999999994, 66.049999999999997, 66.25]
The temp_change is:  6.25
The temp_change is:  3.16
The temp_change is:  2.83
The temp_change is:  4.9
The temp_change is:  4.05
The temp_change is:  5.25
The temp_change is:  6.39
The temp_change is:  6.53
The temp_change is:  4.65
The temp_change is:  3.59
The temp_change is:  1.12
The temp_change is:  1.76
The temp_change is:  1.44
The temp_change is:  0.43

In [50]:
def keywithmaxval(d): 
	return max(d,key=lambda k:d[k])

In [52]:
max_city = keywithmaxval(day_change)

In [53]:
max_city

'Sioux_Falls'

In [59]:
print("The city with the greatest temperature variation between two days is", max_city, 'with a temperate difference of ', str(day_change[max_city]))

The city with the greatest temperature variation between two days is Sioux_Falls with a temperate difference of  5.06


In [ ]:
plt.bar(day_change.keys(), day_change.values())
plt.show()

### Review

####Just as you did with the Citi Bike data, go through the the sequence of temperature readings and calculate the change between days and find the greatest range in high temperatures in the month you measured. Which city had the greatest variation? What is the distribution of the difference? Does the result surprise you? Why or why not?

The greatest change between days was 5.06 in Sioux Falls. The city with the greatest range was Tipton, but with Sioux Falls a close second. Greatest Variation was Sioux Falls.




####Extra

Trying to get the code to loop through and find max range for each column...

### for col in df.columns:
    temp_id = col[:]
    print('The Temp ids are:', temp_id)
    temp_vals = df[col].tolist()
    print('The temp vals are: ',temp_vals)
    temp_change = 0

    for k,v in enumerate(temp_vals):
        max_temp = 0
        min_temp = 200
        if k < len(temp_vals) - 1:
            temp_change = abs(temp_vals[k] - temp_vals[k+1])
            print('The temp_change is: ',temp_change)
    if v > max_temp:
        max_temp = v
        print('max temp equals', max_temp)
    if v < min_temp:
        min_temp = v
        print('min temp equals', min_temp)
    max_range = max_temp - min_temp
    print('The max range is:', max_range)
        
    day_change[(temp_id)] = temp_change 
    print('The day_change is:',day_change)